[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/weaviate-features/model-providers/huggingface/similarity_search_mmBERT.ipynb)

## Similarity search with Multilingual ModernBERT (mmBERT) via Hugging Face and Weaviate

- Read more on the [blog](https://huggingface.co/blog/mmbert)
- Check out the model: [`mmBERT-small`](https://huggingface.co/jhu-clsp/mmBERT-small)

## Dependencies

In [1]:
%%capture
%pip install -q -U weaviate-client

## Connect to Weaviate

This Notebook uses the [`text2vec-transformers`](https://docs.weaviate.io/weaviate/model-providers/transformers/embeddings) module, which is **only** available through Weaviate open-source via [Docker](https://docs.weaviate.io/deploy/installation-guides/docker-installation) or [Kubernetes](https://docs.weaviate.io/deploy/installation-guides/k8s-installation). This integration is **not** available for Weaviate Cloud (WCD) serverless instances, as it requires spinning up a container with the Hugging Face model.

This notebook uses a [pre-built transformers model container](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-transformers#pre-built-images). For this, create a `docker-compose.yml` file with the following contents:
```
---
services:
  weaviate:
    command:
    - --host
    - 0.0.0.0
    - --port
    - '8080'
    - --scheme
    - http
    image: cr.weaviate.io/semitechnologies/weaviate:1.32.2
    ports:
    - 8080:8080
    - 50051:50051
    restart: on-failure:0
    environment:
      TRANSFORMERS_INFERENCE_API: 'http://t2v-transformers:8080'
      QUERY_DEFAULTS_LIMIT: 25
      AUTHENTICATION_ANONYMOUS_ACCESS_ENABLED: 'true'
      PERSISTENCE_DATA_PATH: '/var/lib/weaviate'
      DEFAULT_VECTORIZER_MODULE: 'text2vec-transformers'
      ENABLE_MODULES: 'text2vec-transformers'
      CLUSTER_HOSTNAME: 'weaviate-0'
  t2v-transformers:
    image: cr.weaviate.io/semitechnologies/transformers-inference:jhu-clsp-mmBERT-small
    environment:
      ENABLE_CUDA: '0'
...
``` 
and start up Docker container with
```
docker-compose up -d
```

In [2]:
import weaviate

# Connect to your local Weaviate instance deployed with Docker
client = weaviate.connect_to_local()

print(client.is_ready())

True


In [3]:
# Check the cluster metadata to verify if the module is enabled
client.get_meta()['modules']

{'text2vec-transformers': {'model': {'_name_or_path': 'jhu-clsp/mmBERT-small',
   'add_cross_attention': False,
   'architectures': ['ModernBertForMaskedLM'],
   'attention_bias': False,
   'attention_dropout': 0,
   'bad_words_ids': None,
   'begin_suppress_tokens': None,
   'bos_token_id': 2,
   'chunk_size_feed_forward': 0,
   'classifier_activation': 'gelu',
   'classifier_bias': False,
   'classifier_dropout': 0,
   'classifier_pooling': 'mean',
   'cls_token_id': 1,
   'cross_attention_hidden_size': None,
   'decoder_bias': True,
   'decoder_start_token_id': None,
   'deterministic_flash_attn': False,
   'diversity_penalty': 0,
   'do_sample': False,
   'dtype': 'float32',
   'early_stopping': False,
   'embedding_dropout': 0,
   'encoder_no_repeat_ngram_size': 0,
   'eos_token_id': 1,
   'exponential_decay_length_penalty': None,
   'finetuning_task': None,
   'forced_bos_token_id': None,
   'forced_eos_token_id': None,
   'global_attn_every_n_layers': 3,
   'global_rope_theta': 

## Create a collection
> Collection stores your data and vector embeddings.

In [5]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "MyCollection", and then you need to re-import it again.
import weaviate.classes.config as wc

# Delete the collection if it already exists
if (client.collections.exists("MyCollection")):
    client.collections.delete("MyCollection")

# Create a collection
collection = client.collections.create(
    "MyCollection",
    vector_config=wc.Configure.Vectors.text2vec_transformers(
        # dimensions=384, # default: 384, possible options: 128, 256
    ),
    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Text", data_type=wc.DataType.TEXT),
    ]
)

print("Successfully created collection: MyCollection.")

Successfully created collection: MyCollection.


## Import the Data

In [6]:
# Set up our multi-lingual data
multilingual_data = [
    {"text": "The quick brown fox jumps over the lazy dog."},
    {"text": "Der schnelle braune Fuchs springt über den faulen Hund."},

    {"text": "Artificial intelligence is transforming industries."},
    {"text": "Künstliche Intelligenz verändert Industrien."},

    {"text": "I love to read books in my free time."},
    {"text": "Ich lese gerne Bücher in meiner Freizeit."},

    {"text": "Beautiful weather today, perfect for a walk."},
    {"text": "Schönes Wetter heute, perfekt für einen Spaziergang."},

    {"text": "The capital of Germany is Berlin."},
    {"text": "Die Hauptstadt Deutschlands ist Berlin."}

]


# Insert data objects
response = collection.data.insert_many(multilingual_data)

# Note, the `data` array contains 10 objects, which is great to call insert_many with.
# However, if you have a milion objects to insert, then you should spit them into smaller batches (i.e. 100-1000 per insert)

if (response.has_errors):
    print(response.errors)
else:
    print("Insert complete.")

Insert complete.


Quick check to see if all objects are in.

In [7]:
len(collection)

10

## Query Weaviate: Similarity Search (Text objects)

Similarity search options for text objects in **Weaviate**:

1. [near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

2. [near_object](https://weaviate.io/developers/weaviate/search/similarity#an-object)

3. [near_vector](https://weaviate.io/developers/weaviate/search/similarity#a-vector)

### nearText Example

Find a object in `MyCollection` closest to the query "What's the capital of Germany?". Limit it to only 4 responses.

In [8]:
import weaviate.classes.query as wq
import json

# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
collection = client.collections.get("MyCollection")

query = "What's the capital of Germany?"

response = collection.query.near_text(
    query=query,
    include_vector=True,                             # return the vector embeddings
    return_metadata=wq.MetadataQuery(distance=True), # return the distance
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2))
    print("Vector:", item.vector['default'][:3], '...')
    print("Distance:", item.metadata.distance, "\n")

ID: 126432e1-23d2-4709-aac5-6ccd19552ab1
Data: {
  "text": "The capital of Germany is Berlin."
}
Vector: [-0.002085368847474456, 0.01193431206047535, 0.0018889668863266706] ...
Distance: 0.030832409858703613 

ID: 127ed77a-9508-40c4-962c-04476d21143c
Data: {
  "text": "Beautiful weather today, perfect for a walk."
}
Vector: [-0.0008074266952462494, -0.002204334130510688, 0.007476783357560635] ...
Distance: 0.05185425281524658 

ID: 74862999-97c9-434e-8df7-f2c241a41550
Data: {
  "text": "I love to read books in my free time."
}
Vector: [0.00960230827331543, 0.005805480759590864, -0.0019345288164913654] ...
Distance: 0.052883267402648926 



In [10]:
# Save last item's vector embedding and UUID for near_object and near_vector search
vector = item.vector['default']
uuid = item.uuid
print(len(vector))

384


### nearObject Example

Search through the `JeopardyQuestion` class to find the top 4 objects closest to id (The id was taken from the query above)

In [11]:
print(uuid)

response = collection.query.near_object(
    near_object=uuid, # replace with your id of interest
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties, "\n")

74862999-97c9-434e-8df7-f2c241a41550
ID: 74862999-97c9-434e-8df7-f2c241a41550
Data: {'text': 'I love to read books in my free time.'} 

ID: 127ed77a-9508-40c4-962c-04476d21143c
Data: {'text': 'Beautiful weather today, perfect for a walk.'} 

ID: 1f6493c3-21fa-4893-8093-8101e8f7a616
Data: {'text': 'The quick brown fox jumps over the lazy dog.'} 

ID: f0c7c042-e0ab-4ce3-b725-13a252ac2eda
Data: {'text': 'Ich lese gerne Bücher in meiner Freizeit.'} 



### nearVector Example

Search through the `MyCollection` class to find the top 2 objects closest to the query vector.

In [12]:
print(vector[:5])

response = collection.query.near_vector(
    near_vector=vector, # your vector object goes here
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties, "\n")

[0.00960230827331543, 0.005805480759590864, -0.0019345288164913654, -0.007073240354657173, -0.018555009737610817]
ID: 74862999-97c9-434e-8df7-f2c241a41550
Data: {'text': 'I love to read books in my free time.'} 

ID: 127ed77a-9508-40c4-962c-04476d21143c
Data: {'text': 'Beautiful weather today, perfect for a walk.'} 

ID: 1f6493c3-21fa-4893-8093-8101e8f7a616
Data: {'text': 'The quick brown fox jumps over the lazy dog.'} 

ID: f0c7c042-e0ab-4ce3-b725-13a252ac2eda
Data: {'text': 'Ich lese gerne Bücher in meiner Freizeit.'} 

